<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MMLU_PHI3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Apr 24 01:39:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   35C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q transformers
!pip install -q tqdm
!pip install -q pandas
!pip install -q tensor_parallel
!pip install -q argparse
!pip install -q einops
!pip install -q accelerate
#!pip install -q torch==2.0.0+cu118
!pip install -q torch

!pip install colab-env --upgrade -q
!pip install openai -q

!pip install datasets -q
!pip install utils -q

In [ ]:
!git clone https://github.com/FranxYao/chain-of-thought-hub.git

In [ ]:
import colab_env
import os
import openai
import IPython
import pytz
from datetime import datetime
import json
from pathlib import Path

In [5]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

#model_id = "davidkim205/Rhea-72b-v0.5" # HF AVERAGE = 81.22 #1 APRIL 5TH, 2024

model_id = "microsoft/Phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True, do_sample=True)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [7]:
def generate_answer(prompt):
   outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.1,top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)
   return outputs[0]['generated_text'][len(prompt):].strip()


In [8]:
def test_answer_mmlu_(pred_str, ans):
    pattern = 'the answer is ('
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1][0]
        gold = ans.lower()
        # print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'C'
        # print(ans_str)
        gold = ans.lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

# extract answer in pred_str and compare with ans_str
def test_answer_mmlu_claude_instant(pred_str, ans_str):
    pattern = 'the answer is '
    pred = pred_str.lower().split(pattern)
    if len(pred) == 1:
        return False
    else:
        return pred[1][0] == ans_str.lower()

def test_answer_mmlu_claude(pred_str, ans_str):
    pattern = 'the answer is '
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1]
        for p in pred:
            if(p.isalpha()): break
        pred = p
        print(ans_str)
        gold = ans_str.lower()
        print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'c'
        # print(ans_str)
        gold = ans_str.lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

def test_answer_mmlu(pred_str, ans_str):
    pattern = 'the answer is ('
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1][0]
        gold = ans_str.split('A:\n')[1][0].lower()
        # print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'C'
        # print(ans_str)
        gold = ans_str.split('A:\n')[1][0].lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                # print(am)
                # print(a)
                if(test_answer_mmlu(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:') and not l.startswith("A: Let's think step by step")):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)

    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    # print(am)
    # print(a)
    if(test_answer_mmlu(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [9]:
TASKSTEST0old = [
        'anatomy',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'electrical_engineering',
        'machine_learning',
]


TASKSTEST = [
        'college_computer_science',
        'electrical_engineering',
        'machine_learning',
]

TASKS911 = [
        'anatomy',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'conceptual_physics',
        'econometrics',
        'electrical_engineering',
        'elementary_mathematics',
        'formal_logic',
        'global_facts',
        'high_school_biology',
        'high_school_chemistry',
        'high_school_computer_science',
        'high_school_european_history',
        'high_school_geography',
        'high_school_government_and_politics',
        'high_school_macroeconomics',
        'high_school_mathematics',
        'high_school_microeconomics',
        'high_school_physics',
        'high_school_psychology',
        'high_school_statistics',
        'high_school_us_history',
        'high_school_world_history',
        'public_relations',
        'security_studies',
        'sociology',
        'us_foreign_policy',
        'virology',
        'machine_learning',
        'world_religions']

In [10]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
model_name='PHI3'

In [14]:
%mkdir /content/outputs/
import re
import time
import json

import numpy as np

from tqdm import tqdm
from datasets import load_dataset
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

def main(tasks=TASKSTEST):
    #openai.api_key = openai.api_key
    mmlu_prompt = json.load(open('/content/chain-of-thought-hub/MMLU/lib_prompt/mmlu-cot.json'))
    for task in tasks:

        print()
        print('Testing %s ...' % task)
        print()

        i = 0
        acc = 0
        task_data = load_dataset("lukaemon/mmlu", task, trust_remote_code=True)
        #model="mistral-large-latest"

        #model = "open-mixtral-8x22b"

        with open('/content/outputs/test_%s_%s.txt' % (model_name, task), 'w') as fd:
        #with open('/content/outputs/test_gpt_3.5_turbo_%s.txt' % task, 'w') as fd:
            for q_ in tqdm(task_data['test'], total=len(task_data['test'])):
                q = q_['input'] + '\n'
                for letter in ['A', 'B', 'C', 'D']:
                    q += '(' + letter + ') ' + q_[letter] + ' '
                q += "\nA: Let's think step by step."

                prompt_q = mmlu_prompt[task] + "\n\n" + q
                #print(prompt_q)

                ### ADDED by Frank Morales 23/04/2024
                response=generate_answer(prompt_q)


                #print(response.text)


                ### ADDED by Frank Morales 19/04/2023
                #ans_model = response.text

                try:
                     #x = int(input("Please enter a number: "))
                     #break
                     ans_model = response
                     ans_, residual = extract_ans(ans_model)
                except ValueError:
                     print("Oops!  That was no valid response.  Try again...")
                     print(prompt_q)
                     #print(response)


                del prompt_q
                del response
                ans_, residual = extract_ans(ans_model)

                a = q_['target']
                #print(a)
                fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
                i += 1

                if(test_answer_mmlu_(ans_, a)): acc += 1
            print('%s acc %.4f' % (task, acc / len(task_data['test'])))
        # write accuracy to file
        with open('/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu'), 'a') as fd:
          fd.write('%s acc %.4f\n' % (task, acc / len(task_data['test'])))

    # write average accuracy to file
    acc_list = []
    #with open('/content/outputs/test_%s_%s_acc.txt' % (model, args.prompt_type), 'r') as fd2:

    # with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'multiple'), 'r') as fd2:
    with open('/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu'), 'r') as fd2:
        for line in fd2:
            acc_list.append(float(line.split(' ')[2]))
    with open('/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu'), 'a') as fd:
        fd.write('Average acc %.4f\n' % (np.mean(acc_list)))
    return

if __name__ == '__main__':
    main()

mkdir: cannot create directory ‘/content/outputs/’: File exists

Testing college_computer_science ...



100%|██████████| 100/100 [59:33<00:00, 35.73s/it]


college_computer_science acc 0.4800

Testing electrical_engineering ...



Generating test split:   0%|          | 0/145 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

100%|██████████| 112/112 [1:05:41<00:00, 35.19s/it]

machine_learning acc 0.5536


https://medium.com/@frankmorales_91352/gemini-llm-unveiling-the-future-of-language-models-2eea7aa2c550

MODEL: mistral-large-latest

college_computer_science acc 0.5200
electrical_engineering acc 0.6069
machine_learning acc 0.5982
Average acc 0.5750


MODEL: open-mixtral-8x22b

college_computer_science acc 0.2300
electrical_engineering acc 0.3310
machine_learning acc 0.2321
Average acc 0.2644


MODEL: open-mixtral-8x22b-2404

college_computer_science acc 0.1900
electrical_engineering acc 0.3310
machine_learning acc 0.2232
Average acc 0.2481


MODEL: meta-llama/Meta-Llama-3-8B-Instruct

college_computer_science acc 0.3300
electrical_engineering acc 0.2414
machine_learning acc 0.3125
Average acc 0.2946

MODEL: gemini-pro (trial-1)

college_computer_science acc 0.4600
electrical_engineering acc 0.7241
machine_learning acc 0.4911
Average acc 0.5584

MODEL: gemini-pro (trial-2)

college_computer_science acc 0.4600
electrical_engineering acc 0.6759
machine_learning acc 0.4821
Average acc 0.5393

MODEL: gemini-pro (trial-3)

college_computer_science acc 0.4900
electrical_engineering acc 0.7034
machine_learning acc 0.4911
Average acc 0.5615

MODEL: gemini-1.0-pro-latest (trial-1)

college_computer_science acc 0.4500
electrical_engineering acc 0.6759
machine_learning acc 0.5179
Average acc 0.5479


MODEL: gemini-1.0-pro-latest

college_computer_science acc 0.5300
electrical_engineering acc 0.7172
machine_learning acc 0.5625
Average acc 0.6032

MODEL: PHI3

college_computer_science acc 0.4800
electrical_engineering acc 0.6483
machine_learning acc 0.5536
Average acc 0.5606

In [15]:
#model="mistral-large-latest"
#model = "open-mixtral-8x22b"

print()
print('MODEL: %s'%model_name)
print()
acc_file='/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu')
#print(acc_file)

with open('/content/outputs/test_%s_%s_acc.txt' % (model_name, 'mmlu'), 'r') as fd:
     for line in fd:
            print(line)
            #acc_list.append(float(line.split(' ')[2]))


MODEL: PHI3

college_computer_science acc 0.4800

electrical_engineering acc 0.6483

machine_learning acc 0.5536

Average acc 0.5606

